# 1. Facial Detection with SSD

In [2]:
import zipfile

path = 'data/yalefaces.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('data')
zip_object.close()

In [3]:
import os

print(os.listdir('data/yalefaces/train'))

['subject05.glasses.gif', 'subject08.wink.gif', 'subject05.centerlight.gif', 'subject01.sad.gif', 'subject06.noglasses.gif', 'subject08.centerlight.gif', 'subject11.sleepy.gif', 'subject04.happy.gif', 'subject11.wink.gif', 'subject08.noglasses.gif', 'subject05.wink.gif', 'subject03.sleepy.gif', 'subject15.centerlight.gif', 'subject11.surprised.gif', 'subject06.glasses.gif', 'subject05.happy.gif', 'subject10.normal.gif', 'subject03.centerlight.gif', 'subject04.normal.gif', 'subject09.noglasses.gif', 'subject08.sleepy.gif', 'subject06.rightlight.gif', 'subject14.noglasses.gif', 'subject02.noglasses.gif', 'subject06.centerlight.gif', 'subject09.centerlight.gif', 'subject04.noglasses.gif', 'subject12.glasses.gif', 'subject03.happy.gif', 'subject06.wink.gif', 'subject10.sleepy.gif', 'subject01.rightlight.gif', 'subject12.surprised.gif', 'subject03.surprised.gif', 'subject12.centerlight.gif', 'subject13.happy.gif', 'subject02.surprised.gif', 'subject09.leftlight.gif', 'subject10.leftlight.gi

In [5]:
from PIL import Image

test_image = 'data/yalefaces/train/subject01.leftlight.gif'
image = Image.open(test_image).convert('L')
type(image)

PIL.Image.Image

In [10]:
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow

image_np = np.array(image, 'uint8')

# Colab
# cv2_imshow(image_np)

# Jupyter lab
cv2.imshow("Image", image_np)
# wait for ay key to exit window
cv2.waitKey(0) 
# close all windows
cv2.destroyAllWindows() 

print(image_np.shape)

(243, 320)


In [ ]:
network = cv2.dnn.readNetFromCaffe('model/deploy.prototxt.txt', 'model/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
image = cv2.cvtColor(image_np, cv2.COLOR_GRAY2BGR)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (100, 100)), 1.0, (100,100), (104.0, 117.0, 123.0))
network.setInput(blob)
detections = network.forward()

In [ ]:
conf_min = 0.7
image_cp = image.copy()
for i in range(0, detections.shape[2]):
  confidence = detections[0, 0, i, 2]
  if confidence > conf_min:
    bbox = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (start_x, start_y, end_x, end_y) = bbox.astype('int')
    roi = image_cp[start_y:end_y, start_x:end_x]
    text = "{:.2f}%".format(confidence * 100)
    cv2.putText(image, text, (start_x, start_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0,255,0), 2)
face = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
cv2_imshow(image)
print(image.shape)

In [ ]:
cv2_imshow(face)
print(face.shape)

In [ ]:
face = cv2.resize(face, (60, 80))
cv2_imshow(face)
print(face.shape)

In [ ]:
def detect_face(network, image_path, conf_min = 0.7):
  image = Image.open(image_path).convert('L')
  image = np.array(image, 'uint8')
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
  (h, w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (100, 100)), 1.0, (100, 100), (104.0, 117.0, 123.0))
  network.setInput(blob)
  detections = network.forward()

  face = None
  for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_min:
      bbox = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (start_x, start_y, end_x, end_y) = bbox.astype("int")

      roi = image[start_y:end_y,start_x:end_x]
      roi = cv2.resize(roi, (60, 80))
      cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
      #text = "{:.2f}%".format(confidence * 100)
      #cv2.putText(image, text, (start_x, start_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 2)

      face = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
  return face, image

In [ ]:
test_image = '/content/yalefaces/train/subject02.noglasses.gif'
face, image = detect_face(network, test_image)
cv2_imshow(image)
cv2_imshow(face)

In [12]:
def get_image_data():
  paths = [os.path.join('data/yalefaces/train', f) for f in os.listdir('data/yalefaces/train')]
  #print(paths)
  faces = []
  ids = []
  for path in paths:
    face, image = detect_face(network, path)
    #cv2_imshow(image)
    #cv2_imshow(face)
    #print(path)
    #print(os.path.split(path)[1])
    id = int(os.path.split(path)[1].split('.')[0].replace('subject', ''))
    #print(id)
    ids.append(id)
    faces.append(face)
    cv2_imshow(face)
  return np.array(ids), faces

ids, faces = get_image_data()

NameError: name 'detect_face' is not defined

In [ ]:
ids

In [ ]:
len(ids), len(faces)

In [ ]:
faces[1], faces[1].shape

# 2. Facial Recognition with dlib

In [13]:
path = 'data/celeb_dataset.zip'

zip_object = zipfile.ZipFile(file=path, mode = 'r')
zip_object.extractall('data')
zip_object.close()

In [ ]:
import dlib

face_detector = dlib.get_frontal_face_detector()

def load_training(path_dataset, max_width=400):
  index = {}
  idx = 0
  face_descriptors = None

  subdirs = [os.path.join(path_dataset, f) for f in os.listdir(path_dataset)]

  for subdir in subdirs:
    name = subdir.split(os.path.sep)[-1]
    print(name)
    images_list = [os.path.join(subdir, f) for f in os.listdir(subdir)]
    for image_path in images_list:
      image = Image.open(image_path).convert('RGB')
      image_np = np.array(image, 'uint8')

      if (image_np.shape[1] > max_width):
        image_np = imutils.resize(image_np, width=max_width)

      face_detection = face_detector(image_np, 1)
      for face in face_detection:
        image_np, face_descriptors = extract_descriptor(face, image_np, face_descriptors)

        index[idx] = image_path
        idx += 1
      cv2_imshow(cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))

  return face_descriptors, index

In [ ]:
face_descriptors, index = load_training('data/celeb_dataset/train')

In [ ]:
np.save("dataset_face_descriptors.npy", face_descriptors)
with open("dataset_images_index.pickle", "wb") as f:
  pickle.dump(index, f)

In [ ]:
len(face_descriptors), len(index)

In [ ]:
index